In [181]:
import json
import glob
import os
import numpy as np
from pathlib import Path


In [59]:
#still need to write code that loops through all patient folders
##but for now, just starting with the data in one folder
base_path = '/Users/shrey2/Downloads/PY16N013'
response_path = base_path + '/ResponseInfo/CCEP'
z_path = base_path + '/z-scores'

if len(respInfoPath) > 0:
    print ("You chose %s" % base_path)

#respInfoFile_path = glob.glob(respInfoPath + '/*.json', recursive=True)
#respInfoFiles = os.listdir(respInfoPath)

response_files = os.listdir(response_path)
response_files_path = glob.glob(response_path + '/*.json', recursive=True)

z_files = os.listdir(z_path)
z_files_path = glob.glob(z_path + '/*.json', recursive=True)

You chose /Users/shrey2/Downloads/PY16N013


In [143]:
#example opening dictionary and indexing
k = open(response_files_path[3])
data = json.load(k)
keys = []
for key in data["zscores"]: keys.append(key)
print(keys)

['AIH1_AIH2', 'AIH2_AIH3', 'AIH3_AIH4', 'REF1_REF3', 'MIH1_MIH2', 'MIH2_MIH3', 'MIH3_MIH4', 'PIH1_PIH2', 'PIH2_PIH3', 'PIH3_PIH4', 'LDA1_LDA2', 'LDA2_LDA3', 'LDA3_LDA4', 'LDA4_LDA5', 'LDA5_LDA6', 'LDA6_LDA7', 'LDA7_LDA8', 'LDB1_LDB2', 'LDB2_LDB3', 'LDB3_LDB4', 'LDB4_LDB5', 'LDB5_LDB6', 'LDB6_LDB7', 'LDB7_LDB8', 'LFP1_LFP2', 'LFP2_LFP3', 'LFP3_LFP4', 'LFP4_LFP5', 'LFP5_LFP6', 'LFP6_LFP7', 'LFP7_LFP8', 'LFP8_LFP9', 'LFP9_LFP10', 'LFP10_LFP11', 'LFP11_LFP12', 'LFP12_LFP13', 'LFP13_LFP14', 'LFP14_LFP15', 'LFP15_LFP16', 'LFP17_LFP18', 'LFP18_LFP19', 'LFP19_LFP20', 'LFP20_LFP21', 'LFP21_LFP22', 'LFP22_LFP23', 'LFP23_LFP24', 'LFP25_LFP26', 'LFP26_LFP27', 'LFP27_LFP28', 'LFP28_LFP29', 'LFP29_LFP30', 'LFP30_LFP31', 'LFP31_LFP32', 'LFP33_LFP34', 'LFP34_LFP35', 'LFP35_LFP36', 'LFP36_LFP37', 'LFP37_LFP38', 'LFP38_LFP39', 'LFP39_LFP40', 'LFP41_LFP42', 'LFP42_LFP43', 'LFP43_LFP44', 'LFP44_LFP45', 'LFP45_LFP46', 'LFP46_LFP47', 'LFP47_LFP48', 'LFP49_LFP50', 'LFP50_LFP51', 'LFP51_LFP52', 'LFP52_LFP53',

### respInfoToAdjacencyMatrix
#### could test how the adjacency matrix size changes as we vary (1) threshold and (2) peaks

In [180]:
###now trying to build function for respInfo to adjacency matrix

# INPUTS:
#files = list of file paths
#peaks = string, either n1, n2, p2, or pk2pk (amplitude from n1 to p2)
#threshold = threshold for including the peak in the adjacency matrix

# RETURNS:
#A = adjacency matrix containing response magnitudes of peak of interest
#       specified by input parameter 'peaks'. Stimulating nodes are on the
#       rows, and response nodes are on the columns.
#nodeLabels = the names of the nodes in the adjacency matrix, as a vector
#       in the same order as they appear in 'A'
#allStimInds = vector of nodes indicies that were stimulated

def respInfoToAdjacencyMatrix(files, peaks, threshold):
    responseChs =[];
    peakScores = [];
    stimChs = [];

    #loop over each file containing a stimulation block
    #for (k = 1, k <= length(files), k++):
    for k in files:
        chNames = []
        
        # load info into python dictionary
        fileInfo = k.split("_")
        stimCh1 = fileInfo[2];
        stimCh2 = fileInfo[3];
        #jsonData=jsondecode(fileread(strcat(files(k).folder,"\\",files(k).name)));  
        jsonData = json.load(open(k))
        #chNames = fieldnames(jsonData.zscores);
        for key in jsonData["zscores"]: chNames.append(key)
            
        # loop over each response channel and build lists
        for i in chNames:                                   
            peakValues=[];
            if "pk2pk" in peaks:
                peakAmplitude = jsonData["zscores"][i]["n1"][1] + jsonData["zscores"][i]["p2"][1]              
            else:
                peakAmplitude = abs(jsonData["zscores"][i][peaks][1])
                
            #peakAmplitude from float to int    
            #peakAmplitude = int(peakAmplitude) 
            
   #####Confused by this if statement. original commented out. why are we using len(peakAmplitude)?#####
            #if ((len(peakAmplitude)==1 & (peakAmplitude)>threshold)) | len(peakAmplitude)>1:
            if (peakAmplitude > threshold):
                responseChs.append(i)
                peakScores.append(peakAmplitude)
                stimChs.append(stimCh1 + "_" + stimCh2)
            #   responseChs = [responseChs; chNames(i)];
            #   peakScores = [peakScores; peakAmplitude(end)];
            #   stimChs = [stimChs; {[stimCh1 '_' stimCh2]}];
            
'''
    #Build Adjacency Matrix and Graph
    allStimInds = [];
    nodeLabels = sortrows(chNames);
    A = zeros(length(chNames));

    #loop over each stimulation block
    for i in len(stimChs):

        stimInds = find(any(strcmp(nodeLabels,stimChs[i]),2));
        respInd = find(any(strcmp(nodeLabels,responseChs[i]),2));
        allStimInds.append(stimInds);

        if isempty(respInd):  # if there were no responses, continue
            continue

        # get indicies of all responses that were stimulated in this block
        ind = find(any(strcmp(stimChs,stimChs(i)),2)); 

        # loop over each response and store the zscore in the right spot in the adjacency matrix
        for r in length(respInd):
            r1 = find(strcmp(responseChs(ind),nodeLabels(respInd(r),1)));
            if length(stimInds)<1:
                #1;
                A[stimInds,respInd(r)] = peakScores(ind(r1(end)));
                continue
            #
            if A(stimInds(1),respInd(r)) > 0 & peakScores(ind(r1(end)))>0:
                A[stimInds(1),respInd(r)] = mean([A(stimInds(1),respInd(r)), peakScores(ind(r1(end)))]);
            elif A[stimInds(1),respInd(r)] == 0 & peakScores(ind(r1(end)))>0:
                A[stimInds(1),respInd(r)] = peakScores(ind(r1(end)));

    allStimInds = unique(allStimInds); # only get the unique stimulated pairs


    A=A-diag(diag(A)); # make sure there is no activity on the diagonal
'''
            
respInfoToAdjacencyMatrix(response_files_path, "n1", 0.5)